# Error mitigation

This notebook contains theoretical background and examples for using our Error Mitigation module with Qiskit.

## Theoretical background
Here we describe main theoretical concepts related to mitigation procedure. For more detailed description, see Ref. [0].

### Classical noise model
Let us denote by $\mathbf{M}$ a POVM describing noisy detector and $\mathbf{P}$ denote ideal measurement. In classical noise model, we assume that the relation between the two is given by stochastic, invertible map $\Lambda$:

$$ \mathbf{M} = \Lambda \mathbf{P}.$$

Let us denote by $\mathbf{p}_{exp}$ vector of probabilities obtained on a noisy detector $\mathbf{M}$ measuring any quantum state, and by $\mathbf{p_{ideal}}$ the analogous vector for ideal detector (for the same quantum state). From linearity of the Born's rule, it follows that two vectors are related by the same stochastic map as POVMs:

$$ \mathbf{p_{exp}} = \Lambda \mathbf{p_{ideal}}. $$

Recall that we assumed that $\Lambda$ is invertible. Hence by multiplying last equation by $\Lambda^{-1}$ from both sides, we obtain

$$ \Lambda^{-1} \mathbf{p_{exp}} = \mathbf{p_{ideal}}. $$

Effectively, by this kind of postprocessing, we obtain statistics which we would have obtained on the perfect detector devices.

### Deviations from noise model
#### Effects  on mitigation
Above we assumed a very specific noise model. In practice it is likely that it will not be fulfilled exactly. In such a scenario, we may perform the following decomposition of POVM $M$ describing our device:
$$ \mathbf{M} = \Lambda \mathbf{P} + \mathbf{\Delta}\ ,$$
where $\Delta$ denotes a "coherent" part of the noise and $\Lambda$, as previously, is some stochastic, invertible map. 

In such a case, we can relate probability vector obtained on the noisy detector to ideal one in manner similar as before

$$ \mathbf{p_{exp}} = \Lambda \mathbf{p_{ideal}} + \mathbf{d}\ , $$
where $\mathbf{d}$ denotes a disturbance of probability vector due to existence of coherent part of the noise.
If we now multiply the above expression by inverse of the noise matrix, we obtain

$$ \Lambda^{-1} \mathbf{p_{exp}} = \mathbf{p_{ideal}} + \Lambda^{-1} \mathbf{d}, \  $$


TODO: finish

#### How to choose decomposition?
In general, we can always perform decomposition into "classical" and "coherent" part in infinitely many ways. However, for the ideal detector $P$ modeled as projective measurement in computational basis, there exists a perfectly natural ansatz. Namely, we propose to consider diagonal parts of POVM's $\mathbf{M}$ elements to describe classical part of the noise. 
As a justification for such choice, note that elements of $\mathbf{P}$ for such ideal detector model are diagonal, and the stochastic map would preserve the diagonality. 

Hence, after obtaining description of POVM $\mathbf{M}$ from detector tomography, reconstruction of $\Lambda$ can be achieved by taking only diagonal parts of POVM's elements.




### Finite-size statistics

### When is mitigation succesfull?






#### References
[0] Filip B. Maciejewski, Zoltán Zimborás, Michał Oszmaniec, "Mitigation of readout noise in near-term quantum devices by classical post-processing based on detector tomography", arxiv preprint, https://arxiv.org/abs/1907.08518 (2019)
  



## Mitigating the error using our module

### Performing Quantum Detector Tomography
Our error mitigation approach is based on the knowledge about the noise in the device's detector. Such knowledge can be obtained in procedure known as Quantum Detector Tomography (QDT). To perform QDT, one can follow the steps from our [QDT tutorial](https://github.com/fbm2718/QREM/blob/master/QDT_Tutorial.ipynb). 


In [1]:
import povmtools
import ancillary_functions as anf
import numpy as np

from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer import noise

from quantum_tomography_qiskit import detector_tomography_circuits
from DetectorTomographyFitter import DetectorTomographyFitter


# Choose qubit indices
QDT_qubit_index = [3]

# Select probe kets
QDT_probe_kets = povmtools.pauli_probe_eigenkets

# Generate circuits
QDT_circuits = detector_tomography_circuits(QDT_qubit_index, QDT_probe_kets)

# Get QDT circuits results
backend = Aer.get_backend('qasm_simulator')  #  Get backed
shots_number = 2000  # Define number of measurement repetitions
QDT_job = execute(QDT_circuits, backend=backend, shots=shots_number)
results = QDT_job.result()

# Get ml_povm_estimator using DTF and results
DTF = DetectorTomographyFitter()
ml_povm_estimator = DTF.get_maximum_likelihood_povm_estimator([results], QDT_probe_kets)

## Preparing mitigation

Now that we have the estimator of POVM, we can create QDTErrorMitigator object and prepare it.

In [ ]:
# Creation and preparation of QDTErrorMitigator
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

With prepared mitigator object we gain access to several useful functionalities. For example, we can:
* Correct results of qiskit job by using apply_correction_to_qiskit_job(Results) method.
* Access transition and correction matrices obtained from POVM given during preparation.

In order to properly analyse the results of correction procedure, one have to be aware that in some cases raw application of $\Lambda^{-1}$ to the results may yield quasiprobability (instead of probability) vectors. In such scenario our method calculates closest probability vectors and returns them instead. Distances from raw quasiprobabilities to returned probabilities, can be accessed via distances_from_closest_probability_vector member of mitigator object.

### Error bounds

With access to POVM and the correction and transition matrices, we are able to calculate bounds on several errors. In particular, using povtools module, we can calculate:
* statistical error bound (using get_statistical_error_bound method),
* coherent error bound (using gt_coherent_error_bound method),
* correction error bound (using get_correction_error_bound_from_data or get_correction_error_bound_from_parameters method).  

## Single qubit mitigation scenario

In this section two examples of mitigation for single qubit will be shown. In order to show, that our mitigation scheme is efficient, we first need to create a noisy backend simulator. We start with required imports. 

In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel

import povmtools
from DetectorTomographyFitter import DetectorTomographyFitter
from quantum_tomography_qiskit import detector_tomography_circuits
from QDTErrorMitigator import QDTErrorMitigator

We use standard qiskit methods to create noisy backend simulator. For this tutorial we will create simulator of IBM's Vigo device.

In [2]:
#  What I want to have first is working noisy backend simulation. In order to do that, I will use qiskit noise model
# and download properties of selected backend.

# Build noise model from backend properties.
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_vigo')
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend, why not.
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model.
basis_gates = noise_model.basis_gates

# Finally, get the simulator backend.
simulator_backend = Aer.get_backend('qasm_simulator')

With backend created, what I need to do next is calculating POVM that describes its measurements best. To do that, following our QDT tutorial, I use DetectorTomographyFitter object from our module. In order to do that, I need to create calibration circuits and obtain their results first.

In [3]:
# What I want to do now, is prepare POVM for simulated backend. According to our other notebook, I prepare
# circuits first.

qdt_qubit_index = [0]
qdt_probe_kets = povmtools.pauli_probe_eigenkets
qdt_calibration_circuits = detector_tomography_circuits(qdt_qubit_index, qdt_probe_kets)

# I then execute them on backend prepared earlier.
shots_number = 2000

# Perform a noisy simulation
result = execute(qdt_calibration_circuits, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

# Print counts.
for i in range(len(result.results)):
    print(result.get_counts(i))

{'1': 3, '0': 1997}
{'1': 1949, '0': 51}
{'1': 966, '0': 1034}
{'1': 960, '0': 1040}
{'1': 976, '0': 1024}
{'1': 1010, '0': 990}


Shot's number is obviously a matter of choice in this case. It's preferable to use more (real backends allows more that 8k shots per job) as in that way we obtain better statistics. With more accurate job results, we can -- obviously -- prepare better POVMs. Maximum likelihood POVM calculation is as easy as calling single method now.

In [4]:
# With circuits results I can now use our Detector Tomography Fitter to obtain maximum likelihood POVM estimator.
dtf = DetectorTomographyFitter()
ml_povm_estimator = dtf.get_maximum_likelihood_povm_estimator([result], qdt_probe_kets)

for m_i in ml_povm_estimator:
    print(m_i)

[[ 0.99848906+1.28563720e-19j -0.00150094-8.49319542e-03j]
 [-0.00150094+8.49319542e-03j  0.02532098+1.56396693e-18j]]
[[0.00151094-1.28563720e-19j 0.00150094+8.49319542e-03j]
 [0.00150094-8.49319542e-03j 0.97467902-1.02928532e-17j]]


With POVM calculated, we can now create and prepare mitigator object.

In [6]:
# I can use obtained POVM to correct to prepare mitigation object.
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

With these preparations out of the way, we can now check how efficient our error mitigation approach is. Let's consider two simple scenarios.

### X gate circuit

In this case, we will first create a one qubit circuit (initially in |0><0| state) and we will apply X (not) operation to it. In ideal scenario we would expect all counts in state |1><1|. We begin with circuit creation.

In [7]:
# In order to check how efficient our mitigator is, we need to obtain some noisy data first.
# We will start with preparing simple experiment.

qr = QuantumRegister(1, 'qreg')
cr = ClassicalRegister(1, 'creg')
qc = QuantumCircuit(qr, cr)

qc.x(qr[0])
qc.measure(qr, cr)

Now we execute this circuit on our simulator and check the results.

In [8]:
result = execute(qc, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

for i in range(len(result.results)):
    print(result.get_counts(i))

{'1': 1948, '0': 52}


As we can see, noisy simulator returned some errors (|0><0| counts), as expected. Let's try to correct these results using our mitigator.

In [9]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(corrected_results)

good format
[array([[6.97742498e-04],
       [9.99302258e-01]])]


What we see here, is that instead of counts, our mitigator returned frequencies. In order to obtain counts we could multiply  corrected results times number of shots. This step isn't obviously necessary, as it can be clearly seen that results are indeed better, but let's do it for good measure.

In [14]:
print([corrected_results[0][0][0] * shots_number, corrected_results[0][1][0] * shots_number])

[1.3954849958671827, 1998.6045150041323]


Indeed, we've got barely one count, out of 2000, in calculated in wrong state. Compared to raw 52 counts, this is a very good outcome. Let's try another test.

### H gate circuit

In this case, we will use H (Hadamard) gate, instead of X. Everything else will be exactly like in the first scenario. We begin with creating and executing a circuit and then printing raw results of the job. We expect to obtain equal number of |1> and |0> states conuts.

In [16]:
# Let's try another example.
qr2 = QuantumRegister(1, 'qreg2')
cr2 = ClassicalRegister(1, 'creg2')
qc2 = QuantumCircuit(qr2, cr2)

qc2.h(qr2[0])
qc2.measure(qr2, cr2)

result = execute(qc2, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

for i in range(len(result.results)):
    print(result.get_counts(i))

{'1': 941, '0': 1059}


We see, that we again have got around 60 of wrong counts. Let's try to apply error mitigation.

In [17]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(corrected_results)

good format
[array([[0.51808011],
       [0.48191989]])]


And print them as counts.

In [18]:
print([corrected_results[0][0][0] * shots_number, corrected_results[0][1][0] * shots_number])

[1036.1602121528092, 963.8397878471901]


Once again, the results are better!

There is, however one important thing concerning __Hadamard circuit__ in particular. Due to probabilistic nature of noisy simulation it's possible to obtain perfect (or close to perfect) outcome. In such scenario it's possible for out mitigation procedure to worsen the results. This effect can be observed even in this tutorial, for discussed outcomes of the circuit.